# References:
-

### Import libraries

In [1]:
!pip3 install py2neo

In [2]:
from pprint import pprint
from py2neo import Graph
g = Graph("bolt://localhost:7687/neo4j", password = "test")


In [3]:
query="""
MATCH (a:Amenity) RETURN a.name LIMIT 3
"""
g.run(query).to_data_frame()

,a.name
0,Refrigerator
1,Paid parking off premises
2,Long term stays allowed


In [4]:
query="""
MATCH (a:Listing) WHERE a.price='$150.00' RETURN a.name,a.price,a.url LIMIT 3
"""
g.run(query).to_data_frame()

,a.name,a.price,a.url
0,571029,$150.00,https://www.airbnb.com/rooms/571029
1,729260,$150.00,https://www.airbnb.com/rooms/729260
2,733654,$150.00,https://www.airbnb.com/rooms/733654


### Content-based filtering

In [5]:

query = """
MATCH(u:User{name:'10952'})-[:RATED]->(s:Listing)-[:HAS_AMENITY]->(c:Amenity)<-[:HAS_AMENITY]-(z:Listing)
WHERE NOT EXISTS ((u)-[:RATED]->(z))
WITH s, z, COUNT(c) AS intersection
MATCH (s)-[:HAS_AMENITY]->(sc:Amenity)
WITH s, z, intersection, COLLECT(sc.name) AS s1
MATCH (z)-[:HAS_AMENITY]->(zc:Amenity)
WITH s, z, s1, intersection, COLLECT(zc.name) AS s2
WITH s, z, intersection, s1+[x IN s2 WHERE NOT x IN s1] AS union, s1, s2
RETURN s.name as UserListing, z.name as Recommendate, s1 as UserListingAmenities, s2 as RecommendateListingAmenities, ((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC;
"""
g.run(query).to_data_frame()

,UserListing,Recommendate,UserListingAmenities,RecommendateListingAmenities,jaccard
0,2818,17968731,"[Ethernet connection, Dedicated workspace: tab...","[Ethernet connection, Hot water kettle, Essent...",0.600000
1,2818,2211896,"[Ethernet connection, Dedicated workspace: tab...","[Babysitter recommendations, Dedicated workspa...",0.547170
2,2818,9104541,"[Ethernet connection, Dedicated workspace: tab...","[Ethernet connection, Essentials, Private entr...",0.540000
3,2818,43978539,"[Ethernet connection, Dedicated workspace: tab...","[Pour-over coffee, Hot water kettle, Essential...",0.530612
4,2818,17968930,"[Ethernet connection, Dedicated workspace: tab...","[Ethernet connection, Hot water kettle, Essent...",0.520833
...,...,...,...,...,...
5390,2818,50916897,"[Ethernet connection, Dedicated workspace: tab...","[TV, Kitchen, Paid parking on premises, Washer...",0.022222
5391,2818,778820,"[Ethernet connection, Dedicated workspace: tab...","[Cable TV, TV with standard cable, Kitchen, He...",0.021739
5392,2818,51779734,"[Ethernet connection, Dedicated workspace: tab...","[TV, Dedicated workspace, Kitchen, Long term s...",0.021739
5393,2818,53043715,"[Ethernet connection, Dedicated workspace: tab...","[Free parking on premises, TV, Kitchen, Long t...",0.021277


In [6]:
cid = "8726758"
query = """
        MATCH (p1:User {name:'8726758'})-[x:RATED]->(m:Listing)<-[y:RATED]-(p2:User) WITH COUNT(m) AS numbershows, SUM(x.rating * y.rating) AS xyDotProduct,SQRT(REDUCE(xDot = 0.0, a IN COLLECT(x.rating) | xDot + a^2)) AS xLength,SQRT(REDUCE(yDot = 0.0, b IN COLLECT(y.rating) | yDot + b^2)) AS yLength,p1, p2 RETURN p1.name as SelectedUser, p2.name as SimilarUser;
        """

# g.run(query).to_data_frame()
neighbors = {}
for i in g.run(query, cid = "8726758", k = 5).data():
    print(i)

{'SelectedUser': '8726758', 'SimilarUser': '303616415'}
{'SelectedUser': '8726758', 'SimilarUser': '409078673'}
{'SelectedUser': '8726758', 'SimilarUser': '56663354'}
{'SelectedUser': '8726758', 'SimilarUser': '330320508'}
{'SelectedUser': '8726758', 'SimilarUser': '47472820'}
{'SelectedUser': '8726758', 'SimilarUser': '15428796'}
{'SelectedUser': '8726758', 'SimilarUser': '303383524'}
{'SelectedUser': '8726758', 'SimilarUser': '156086800'}
{'SelectedUser': '8726758', 'SimilarUser': '61906534'}
{'SelectedUser': '8726758', 'SimilarUser': '271440120'}
{'SelectedUser': '8726758', 'SimilarUser': '245214095'}
{'SelectedUser': '8726758', 'SimilarUser': '95511875'}
{'SelectedUser': '8726758', 'SimilarUser': '275958549'}
{'SelectedUser': '8726758', 'SimilarUser': '62950521'}
{'SelectedUser': '8726758', 'SimilarUser': '157035237'}
{'SelectedUser': '8726758', 'SimilarUser': '262880880'}
{'SelectedUser': '8726758', 'SimilarUser': '220128053'}
{'SelectedUser': '8726758', 'SimilarUser': '250234578'

In [29]:
neighbors

{}